In [ ]:
#Airbnb Munich dataset from 24th of December, 2020 - see: http://insideairbnb.com/get-the-data.html
import pandas as pd

calendar_df = pd.read_csv("./Data/calendar.csv")
listings_df = pd.read_csv("./Data/listings.csv")

In [ ]:
###########################################################################################################
# Question 1: In which months are most Airbnb listings still available (total and by room type)?
###########################################################################################################

In [ ]:
calendar_df.head()

In [ ]:
# Select relevant columns 
calendar_df = calendar_df[["listing_id", "date", "available"]]
calendar_df.head()

In [ ]:
calendar_df.dtypes

In [ ]:
print("We changed the type of the two columns date and available:")
calendar_df['date'] = pd.to_datetime(calendar_df['date'], errors='coerce')
calendar_df["available"] = calendar_df["available"].replace({'t': True, 'f': False})
calendar_df.dtypes

In [ ]:
print("Please find some important data for the calender dataframe:")
print("- The number of rows is: " + str(f"{len(calendar_df.index):,}"))
print("- The number of Nan values for each column is 0:")
print(calendar_df.isnull().sum())

In [ ]:
print("The Airbnb Munich dataset originally includes data between " + min(calendar_df['date']).strftime("%Y/%m/%d") + " and " + max(calendar_df['date']).strftime("%Y/%m/%d") +".")
print("As we are only interested in data from 2021, we skip the December 2020 data.")
calendar_df = calendar_df.loc[(calendar_df["date"] >= "2021-01-01") & (calendar_df["date"] <= "2021-12-31")]
print("The Airbnb Munich dataset now includes data between " + min(calendar_df['date']).strftime("%Y/%m/%d") + " and " + max(calendar_df['date']).strftime("%Y/%m/%d") +".")

In [ ]:
print("The number of rows is: " + str(f"{len(calendar_df.index):,}"))

In [ ]:
# number of listings per month:
no_listings_per_month=calendar_df[['listing_id']].groupby([calendar_df['date'].dt.year.rename('year'), calendar_df['date'].dt.month.rename('month')]).count()
no_listings_per_month.rename(columns={'listing_id':'number_of_listings'}, inplace=True)
print (no_listings_per_month)
print( "\n" + "The graphic shows that there are less rooms listed for some months.") 
print("December is the month with the lowest amount of rooms listed at Airbnb - Keep in mind that we also have less data provided for that month (data for the 30th and 31st of December is missing)")

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl


x = [u'Jan', u'Feb', u'Mar', u'Apr', u'May', u'Jun', u'Jul', u'Aug', u'Sep', u'Oct', u'Nov', u'Dec']
y = no_listings_per_month.transpose().iloc[::3, :].values[0]

fig, ax = plt.subplots() 

fig.set_size_inches(16, 6)
width = 0.75 # the width of the bars 
ind = np.arange(len(y))  # the x locations for the groups

ax.barh(ind, y, width, color="blue")
ax.set_yticks(ind+width/2)
ax.set_yticklabels(x, minor=False)
for i, v in enumerate(y):
    ax.text(v, i, str(format(v, ',')), color='black')

plt.title('Number of Airbnb listings per month', fontsize=14)
plt.xlabel('Number of listings')
plt.ylabel('Month')      
ax.xaxis.set_major_formatter(mpl.ticker.FuncFormatter(lambda x, p: format(int(x), ',')))


plt.show()

In [ ]:
# merge the two dataframes listings_df and calendar_df 
calendar_df_with_room_types=pd.merge(calendar_df,listings_df, left_on='listing_id',right_on='id')
# select relevant columns
calendar_df_with_room_types = calendar_df_with_room_types[["date", "available", "listing_id", "room_type"]]

print("Question 1a: In which months are most Airbnb listings still available (total)?", end="\n\n")
sum_1a = calendar_df_with_room_types.groupby([calendar_df_with_room_types['date'].dt.year.rename('year'), calendar_df_with_room_types['date'].dt.month.rename('month'), 'available'])['listing_id'].sum()
result_1a = (sum_1a / sum_1a.groupby(l
                                     
                                     
                                     evel=[0, 1]).transform("sum")*100)
result_1a_df = result_1a.to_frame().rename(columns = {'listing_id':'percentage'}) #Convert series to dataframe and rename column
print(result_1a_df, end="\n\n")
print("February 2021 is the month with the highest availablity, followed by March 2021 and January 2021. This makes sense,")
print("if you think of the current Corona situation. People hesitate to book a room soon.")
print("October 2021 is the busiest month regarding the availablity rate. People look forward to go to the Octoberfest again.")


print("Question 1b: In which months are most Airbnb listings still available (by room_type)?", end="\n\n")
sum_1b = calendar_df_with_room_types.groupby([calendar_df_with_room_types['date'].dt.year.rename('year'), calendar_df_with_room_types['date'].dt.month.rename('month'), 'available', 'room_type'])['listing_id'].sum()
result_1b = sum_1b / sum_1b.groupby(level=[0, 1]).transform("sum")*100
result_1b_df = result_1b.to_frame().rename(columns = {'listing_id':'percentage'}) #Convert series to dataframe and rename column
pd.set_option('display.max_rows', 1000)
print(result_1b_df, end="\n\n")
print("This data is hard to read. Let's split the data and make some nice graphics and try out two different python libaries.")


In [ ]:
# The final dateframes from here also serve as input for the second graphic (see below)
result_1b_df_ind = result_1b_df.reset_index()

print("Percentage of available rooms by month and room type:", end="\n\n")
result_1b_available_df_ind= result_1b_df_ind[result_1b_df_ind['available'] == True]
available_df = result_1b_available_df_ind.pivot('month', 'room_type', 'percentage')
print(available_df, end="\n\n\n\n")

print("Percentage of unavailable rooms by month and room type:", end="\n\n")
result_1b_unvailable_df_ind= result_1b_df_ind[result_1b_df_ind['available'] == False]
unavailable_df = result_1b_unvailable_df_ind.pivot('month', 'room_type', 'percentage')
print(unavailable_df, end="\n\n\n\n")


print("Entire home/apartments, private rooms and shared rooms have the hightest availability rate in Jan, Feb and Mar 2021.")
print("Only hotel rooms are more booked in January 2021 compared to the other months.")

#Hint: the sum of values from the first row of the two graphhics is 100 (equivalent for all other rows)
#first row: 33.857044 + 0.897881 + 31.324853 + 1.372434 + 21.252804 + 0.512711 + 10.579063 + 0.203210 = 100 

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.font_manager import FontProperties
import numpy as np

# Set the labels
fig = plt.figure()
fig.suptitle('Availability rate of Airbnb listings for 2021', fontsize=14)
plt.xlabel('Month', fontsize=10)
plt.ylabel('Percentage', fontsize=10)

# Transform the data 
result_1a_df_ind = result_1a_df.reset_index()
available = np.array(result_1a_df_ind[result_1a_df_ind['available'] == True].iloc[:,[3]].T.values[0])  # Transpose (.T)
unavailable = np.array(result_1a_df_ind[result_1a_df_ind['available'] == False].iloc[:,[3]].T.values[0])

# Make the stacked bar plot
columns = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
pos = np.arange(len(columns))
p1 = plt.bar(pos, np.add(unavailable , available), color='red', edgecolor='red', label= 'unavailable')
p2 = plt.bar(pos, available, color='green', edgecolor='green', label= 'available')
plt.xticks(pos, columns)

# Create a legend
fontP = FontProperties()
fontP.set_size('medium')
plt.legend(handles=[p1, p2], bbox_to_anchor=(1.05, 1), loc='upper left', prop=fontP)

# Show graphic
plt.show()

In [ ]:
import altair as alt
import calendar

# Change from pivot table to normal table format; add column available
def prepare_df(df, name):
    df = df.stack().reset_index()
    df.columns = ['month', 'room_type', 'values']
    df['available'] = name
    return df

# Settings for displaying the chart
months = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
order = ['available', 'unavailable'] 

# Prepare the data (dateframes see two cells above)
unavailable_df = prepare_df(unavailable_df, 'unavailable')
available_df = prepare_df(available_df, 'available')
df = pd.concat([available_df, unavailable_df]) # join the two dataframes
df['month'] = df['month'].apply(lambda x: calendar.month_abbr[x]) # convert values from month column to month name


# Make the stacked bar plot
chart = alt.Chart(df, title=['Availability rate of different Airbnb room types', '   ']).mark_bar().encode(

    # Use room type as x-axis 
    x=alt.X('room_type:N', title=None),

    # Use percentage as y-axis 
    y=alt.Y('sum(values):Q',
        axis=alt.Axis(
            grid=False,
            title='Percentage')),

    # Use month as the set of columns to be represented in each group
    column=alt.Column('month:N',  title=None, sort=months),

    # Set the colours 
    color=alt.Color('available:N', sort=order, 
            scale=alt.Scale(
                # make it look pretty with an enjoyable color pallet
                range=['green', 'red']
            ), title =None, 
        ), 
    # Determine the order for displaying (we want available on the bottom)
    order="order:Q" 
    )

#Configure the chart and display it
chart.configure_view(strokeOpacity=0)

In [ ]:
###########################################################################################################
# Question 2: In which Munich areas (zip codes) are the best Airbnb listings (according to total rating)?
###########################################################################################################

In [ ]:
pd.set_option('display.max_columns', 100)
listings_df.head()

In [ ]:
# Select relevant columns 
# Do not use neighboorhood_cleansed since there is no geo data available for free regarding the districts of Munich

listings_data_df = listings_df[["id", "latitude", "longitude", "review_scores_rating"]]
listings_data_df.head()

In [ ]:
listings_data_df.dtypes

In [ ]:
print("- The number of rows is: " + str(f"{len(listings_data_df.index):,}"))

In [ ]:
# Use latitude and longitude as input to get the zip code of the Airbnb listing via geopandas
import geopandas as gpd
from shapely.geometry import Point

# Source for map data: https://www.suche-postleitzahl.org/plz-karte-erstellen
gdf_locations = gpd.read_file('./Data/plz-5stellig.shp', dtype={'plz': str})
# Data transformation
listings_data_df = listings_data_df.copy()
listings_data_df['loc'] = listings_data_df.apply(lambda x: list([x['latitude'], x['longitude']]),axis=1).copy()
listings_data_df['loc'] = listings_data_df['loc'].apply(lambda x: Point(x[1],x[0]))
# Retrieving map data
geo_data = gpd.GeoDataFrame(listings_data_df,geometry=listings_data_df['loc'],crs=gdf_locations.crs).drop(['loc'], axis=1)
geo_result = gpd.sjoin(geo_data,gdf_locations,how='left',op='within')
geo_result.head()

In [ ]:
# Convert geodataframe to pandas-dataframe
munich_df  = pd.DataFrame(geo_result)
# Remove non-relevant columns
munich_df = munich_df.drop(columns=['index_right',  'geometry', 'latitude', 'longitude'])
munich_df.head()

In [ ]:
print("- The number of rows is: " + str(f"{len(munich_df.index):,}"))

In [ ]:
# Remove null values 
print(munich_df.isnull().sum())
munich_df = munich_df.dropna()

In [ ]:
#Display amount of the different review_scores for each zip code
munich_df.groupby(['plz','review_scores_rating']).size().unstack()

In [ ]:
# Calculate mean review_score for each zip code
res_2 = munich_df.groupby(['plz']).agg(['count','mean'])
res_2  = res_2 .drop(columns=['id'])
res_2 = res_2.reset_index()
print(res_2.head(100))
print()
print("We can see that the ratings are very high for all listings (all above 90).")
print("Some of the zip codes only have a few ratings.")

In [ ]:
# Get polygon geometry data for the plot
pd.set_option('display.max_rows', 100)
plot_input_data = pd.merge(left=res_2, right=gdf_locations, on='plz', how='left')
plot_input_data = plot_input_data.drop(columns=['einwohner', 'note', 'qkm'])
plot_input_data.head(100)

In [ ]:
from geopandas import GeoDataFrame
# Create the plot
plt.style.use('seaborn')
%matplotlib inline
fig, ax = plt.subplots(figsize=(28,14))

GeoDataFrame(plot_input_data).plot(ax=ax, column=plot_input_data.columns[3], categorical=False, legend=True, cmap='summer_r',
                                   missing_kwds=dict(color='grey'))

#Set the title
ax.set_title('Average rating of Airbnb listings for each zip code', pad=10, fontsize=18)

# Remove axis labels for latitude and longitude
ax.axes.xaxis.set_visible(False)
ax.axes.yaxis.set_visible(False)
ax.set(facecolor='lightgrey');

# Add zip code labels
plot_input_data.apply(lambda x: ax.annotate(text=x.plz, xy=x.geometry.centroid.coords[0], ha='center'),axis=1);

print("Annotation: Zip codes with less than 10 ratings were excluded from the analysis and marked in dark gray.")
print("Results: It seems to be that most of the popular areas are placed in the West of Munich.")
print("This area has very good traffic connections to the inner city and also to the Oktoberfest.")
print("As some the zip codes in the West of Munich like Laim have lower house/appartment prices compared to other areas,") 
print("let's investigate the relationship between listing price and listing (total) rating")

In [ ]:
# same plot with all Airbnb listings included in the figure

from geopandas import GeoDataFrame
# Create the plot
plt.style.use('seaborn')
%matplotlib inline
fig, ax = plt.subplots(figsize=(28,14))

GeoDataFrame(plot_input_data).plot(ax=ax, column=plot_input_data.columns[3], categorical=False, legend=True, cmap='summer_r',
                                   missing_kwds=dict(color='grey'))

listings_data_df['const']=1
GeoDataFrame(listings_data_df).plot(ax=ax, column='const', categorical=False, legend=False, cmap='gray');

#Set the title
ax.set_title('Average rating of Airbnb listings for each zip code (including the locations of the listings)', pad=10, fontsize=18)

# Remove axis labels for latitude and longitude
ax.axes.xaxis.set_visible(False)
ax.axes.yaxis.set_visible(False)
ax.set(facecolor='lightgrey');

print("Most of the Airbnb listings are located near to the inner city.")

In [ ]:
###########################################################################################################
# Question 3: Do cheaper listings have a lower rating compared to more expensive listings? 
###########################################################################################################


In [ ]:
# Data selection
listings_data_df_price = listings_df[["id", "price", "review_scores_rating"]]
listings_data_df_price.head()

In [ ]:
# Data cleaning of coloumn price
listings_data_df_price["price"] = listings_data_df_price["price"].str.replace("$", '',regex=True).str.replace(",", '',regex=True).astype(float)
listings_data_df_price.head()

In [ ]:
listings_data_df_price.dtypes

In [ ]:
listings_data_df_price.isnull().sum()
# As NaN values are not taken into account in the scotter plot, we do not need to remove them.

In [ ]:
print(listings_data_df_price[["price", "review_scores_rating"]].describe())
print()
print("The high standard deviation (247) indicates that the data are more spread out.")
print("Also the maximum is very high (8255).")
print("There is also at least one price value, which is equals 0 (see minimum).")

In [ ]:
# Let's have a look at the listings with a price of 0. 
# All rows with price values of 0, also have NaN rating values.
# As NaN values are not taken into account in the scotter plot, we do not need to remove them.
listings_data_df_price[listings_data_df_price["price"] == 0]

In [ ]:
# Scatter plot to describe relationship between price and (total) rating
import matplotlib.pyplot as plt
ax = listings_data_df_price.plot(kind='scatter', x='price', y='review_scores_rating')

plt.xlabel('Price in $')
plt.ylabel('Total review score')
plt.title('Scatter plot of total review rating vs. price')

ax.xaxis.set_major_formatter(mpl.ticker.StrMethodFormatter('{x:,.0f}'))
plt.show()

print("From the scatter plot, we can see the outliers.")

In [ ]:
# Remove outliers by querying the rows with a price less than $600
price_less_than_600 = listings_data_df_price.query('price <= 600')
count_outliers = len(listings_data_df_price[listings_data_df_price['price'] > 600])
percentage_outliers = len(listings_data_df_price[listings_data_df_price['price'] > 600]) / listings_data_df_price.shape[0] * 100
print('We found {} rows ({:.2f}%) with a price less than $600.'.format(count_outliers, percentage_outliers))

In [ ]:
import matplotlib.pyplot as plt
plt.scatter(price_less_than_600['price'],price_less_than_600['review_scores_rating'])
plt.xlabel('Price in $')
plt.ylabel('Total review score')
plt.title('Scatter plot of total review rating vs. price (price less than $600)')
plt.show()

print("From the scatter plot, we can see that there is hardly no relationship between total review score and price.")
print("Many listings in the lower price range also have very high ratings.")

In [ ]:
import seaborn as sb

pearsoncorr = price_less_than_600[["price", "review_scores_rating"]].corr(method='pearson')

sb.heatmap(pearsoncorr, 
            xticklabels=pearsoncorr.columns,
            yticklabels=pearsoncorr.columns,
            cmap='RdBu',
            annot=True,
            linewidth=0.5)
print("From the results of the correlation coefficient, we come to the same conclusion:")
print("The correlation coefficient of -0.011 is considered as negligible correlation.")